In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../src')

from dataset import DatasetConfig, DatasetBuilder
import os
from functools import partial
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import defaultdict, namedtuple
from typing import Callable, Dict, List, Tuple
%matplotlib inline

In [2]:
ds_config =  DatasetConfig(model_dir='../dataset/pinecone/sparse/0/',
                           images_dir='../dataset/pinecone/images/',
                           batch_from_single_image=True,
                           batch_size=8)

In [3]:
dataset_builder = DatasetBuilder(ds_config)
train_ds = dataset_builder.build_train_dataset()

In [4]:
test_ds = dataset_builder.build_test_dataset()

In [4]:
lines = []
colors = []

def make_line(origin, direction, length=10):
    return origin, origin + direction * length

for elements in train_ds.take(200).as_numpy_iterator():
    origins = elements.origins
    directions = elements.directions
    pixels = elements.colors
    N = origins.shape[0]
    for i in range(N):
        p0, p1 = make_line(origins[i, ...], directions[i, ...])
        lines.append(p0)
        lines.append(p1)
        colors.append(pixels[i, ...])

In [8]:
elements

ColoredRays(origins=array([[-1.3584163,  2.8891244, -0.7654388],
       [-1.3584163,  2.8891244, -0.7654388],
       [-1.3584163,  2.8891244, -0.7654388],
       [-1.3584163,  2.8891244, -0.7654388],
       [-1.3584163,  2.8891244, -0.7654388],
       [-1.3584163,  2.8891244, -0.7654388],
       [-1.3584163,  2.8891244, -0.7654388],
       [-1.3584163,  2.8891244, -0.7654388]], dtype=float32), directions=array([[ 0.21775223,  0.36321905,  0.9058935 ],
       [ 0.48083252,  0.4632607 ,  0.74443144],
       [ 0.67416   ,  0.4551204 ,  0.5816901 ],
       [ 0.19054098,  0.37953192,  0.9053382 ],
       [ 0.5216486 ,  0.24016047,  0.8186518 ],
       [ 0.5905296 , -0.33727977,  0.7331424 ],
       [ 0.46349207,  0.44260776,  0.7676338 ],
       [ 0.3815262 , -0.24660194,  0.89084554]], dtype=float32), colors=array([[0.69411767, 0.6745098 , 0.69803923],
       [0.26666668, 0.30980393, 0.18431373],
       [0.30588236, 0.3137255 , 0.3019608 ],
       [0.654902  , 0.65882355, 0.6666667 ],
    

In [7]:
data = np.asarray(lines)
np.save('lines.npy', data)

colors = np.asarray(colors)
np.save('colors.npy', colors)

In [9]:
element

Point(x=<tf.Tensor: shape=(), dtype=int32, numpy=9>, y=<tf.Tensor: shape=(), dtype=int32, numpy=9>)